In [1]:
from thrift4DL.client import VisionClient
import numpy as np
from zaailabcorelib.ztools import encode_image, load_image

In [2]:
client = VisionClient(host='127.0.0.1', port='9090')

In [3]:
img_arr = load_image('../example/mnist/num5.png')
img_hex = encode_image(img_arr)

In [4]:
result = client.predict(img_hex)

In [5]:
result

TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}')

In [6]:
client.ping()

In [7]:
# %timeit client.predict(img_hex)

In [8]:
from ailabtools.ailab_multiprocessing import pool_worker

In [10]:
%timeit -n 1 -r 5 ret = pool_worker(target=client.predict, inputs=[img_hex]*3000, num_worker=10)

100%|██████████| 3000/3000 [00:05<00:00, 513.46it/s]


6.05 s ± 1.59 s per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [8]:
5.61/3000

0.0018700000000000001

In [15]:
1/0.0018700000000000001

534.75935828877

In [43]:
ret = pool_worker(target=client.predict, inputs=[img_hex]*1000, num_worker=20)

100%|██████████| 1000/1000 [00:01<00:00, 723.44it/s]


In [44]:
len(ret)

1000

In [45]:
ret

[TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_message='Successful', response='{"pred_num": 5, "pred_score": 0.9998825788497925}'),
 TVisionResult(error_code=0, error_messa

In [ ]:
0.4

In [34]:
result = client.predict(img_hex + "gg")

In [35]:
result

TVisionResult(error_code=-1, error_message='Internal Error', response='')